In [12]:
import cv2
import numpy as np
from scipy.cluster.hierarchy import fclusterdata
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt


In [ ]:
#dilation
kernel = np.ones((5,150), np.uint8)
dilated = cv2.dilate(cleaned_image, kernel, iterations = 1)
plt.imshow(dilated, cmap='gray');

(contours, heirarchy) = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
sorted_contours_lines = sorted(contours, key = lambda ctr : cv2.boundingRect(ctr)[1]) # (x, y, w, h)


img2 = image.copy()
line_images = []


for ctr in sorted_contours_lines:
    
    x,y,w,h = cv2.boundingRect(ctr)
    cv2.rectangle(img2, (x,y), (x+w, y+h), (255, 100, 250), 2)

     # Crop each line and store it in the line_images array
    line_image = img[y:y + h, x:x + w]
    line_images.append(line_image)
   
plt.imshow(img2);

img3 = img.copy()
words_list = []

for line in sorted_contours_lines:
    
    # roi of each line
    x, y, w, h = cv2.boundingRect(line)
    roi_line = dilated2[y:y+w, x:x+w]
    
    # draw contours on each word
    (cnt, heirarchy) = cv2.findContours(roi_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    sorted_contour_words = sorted(cnt, key=lambda cntr : cv2.boundingRect(cntr)[0])
    
    for word in sorted_contour_words:
        
        if cv2.contourArea(word) < 400:
            continue
        
        x2, y2, w2, h2 = cv2.boundingRect(word)
        words_list.append([x+x2, y+y2, x+x2+w2, y+y2+h2])
        cv2.rectangle(img3, (x+x2, y+y2), (x+x2+w2, y+y2+h2), (255,255,100),2)
        
plt.imshow(img3);

In [ ]:
import math 

sorted_centroids = sorted(centroids, key=lambda c: (c[1], c[0]))
c1 = sorted_centroids[0]    
distance = 1000  # Initialize with a large value
closest_c2 = None  # To store the second centroid of the closest pair
largest_distance = 0  # This will sum all the distances
num_distances = 0
for i in range(len(centroids) - 1):  
    c2 = sorted_centroids[i + 1]  # Neighboring centroid
    
    # Calculate the Euclidean distance between the two centroids
    dis = math.sqrt((c2[0] - c1[0])**2 + (c2[1] - c1[1])**2)
    
    
    # Update the minimum distance and the closest pair of centroids
    if dis < distance:
        distance = dis
        closest_c2 = c2 
    else:
        largest_distance = dis
    num_distances+=1
 

print(largest_distance)
print(f"Minimum distance between centroid {c1} and centroid {closest_c2} is: {distance}")

# Draw a line between the two closest centroids
cv2.line(output_image2, c1, closest_c2, (255, 255, 200), 2)  # Green line between c1 and closest_c2

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image2, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Image with Closest Centroids and Line")
plt.show()

In [ ]:

distance_threshold = 30
labels = fclusterdata(np.array(centroids), t=distance_threshold, criterion="distance")


clusters = {}
for i, label in enumerate(labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(contours[i])
    


segments = []
margin = 10

for cluster_contours in clusters.values():
    # Get bounding box
    x_min = min([cv2.boundingRect(c)[0] for c in cluster_contours]) - margin
    y_min = min([cv2.boundingRect(c)[1] for c in cluster_contours]) - margin
    x_max = max([cv2.boundingRect(c)[0] + cv2.boundingRect(c)[2] for c in cluster_contours]) + margin
    y_max = max([cv2.boundingRect(c)[1] + cv2.boundingRect(c)[3] for c in cluster_contours]) + margin

    # Crop the image around the bounding box
    segment = image[y_min:y_max, x_min:x_max]
    segments.append(segment)
    
    # Draw rectangle around the cluster
    cv2.rectangle(output_image2, (x_min, y_min), (x_max, y_max), (0, 255, 0), 1)


plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image2, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Braille Image with Clusters Highlighted")
plt.show()


num_segments = len(segments)
images_per_row = 10
# Calculate the number of rows required
num_rows = (num_segments // images_per_row) + (num_segments % images_per_row != 0)
# Create a figure
plt.figure(figsize=(3 * images_per_row, 3 * num_rows))

# Plot each segment
for i, segment in enumerate(segments):

    row = i // images_per_row  
    col = i % images_per_row
    plt.subplot(num_rows, images_per_row, i + 1)
    plt.imshow(segment, cmap='gray')
    plt.axis('off')
    plt.title(f'Segment {i+1}')

# Adjust layout and show the images
plt.tight_layout()
plt.show()

In [ ]:
for cX, cY in centroids:
    # Draw horizontal line passing through the dot
    cv2.line(output_image, (0, cY), (output_image.shape[1], cY), (0, 255, 0), 1)
    # Draw vertical line passing through the dot
    cv2.line(output_image, (cX, 0), (cX, output_image.shape[0]), (0, 255, 0), 1)

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Braille Image with Lines through Dots")
plt.show()